In [20]:
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ayyuce/vegetables")

print("Path to dataset files:", path)


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Path to dataset files: C:\Users\Aarin Bhatta\.cache\kagglehub\datasets\ayyuce\vegetables\versions\1


In [4]:
%pip install ultralytics

  Using cached ultralytics-8.3.226-py3-none-any.whl.metadata (37 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
  Using cached polars-1.35.1-py3-none-any.whl.metadata (10 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/114.6 kB ? eta -:--:--
     ------------------------------- ------- 92.2/114.6 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 114.6/114.6 kB 1.7 MB/s eta 0:00:00
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
     ---------------------------------------- 0.0/60.8 kB ? e

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from ultralytics import YOLO

# 3. Loading a pre-trained model (yolov8n is the smallest)
model = YOLO('yolov8n.pt')

!wget -q https://ultralytics.com/images/bus.jpg
results = model.predict('bus.jpg')

# 5. Showing the results (this will display the image in Colab)
results[0].show()

'wget' is not recognized as an internal or external command,
operable program or batch file.



image 1/1 c:\Users\Aarin Bhatta\OneDrive\Desktop\Project\CulinaryVisAI\CulinaryVis\cv_model\venv\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 39.2ms
Speed: 1.6ms preprocess, 39.2ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 480)


In [22]:
import os
import glob
import random
import os.path as osp
import shutil  # <-- IMPORT SHUTIL FOR COPYING

# --- 1. DEFINE YOUR SOURCE PATHS (This looks correct) ---
source_img_dir = "C:/Users/Aarin Bhatta/.cache/kagglehub/datasets/ayyuce/vegetables/versions/1/dataset/images"
source_lbl_dir = "C:/Users/Aarin Bhatta/.cache/kagglehub/datasets/ayyuce/vegetables/versions/1/dataset/labels"

# --- 2. DEFINE YOUR NEW DESTINATION FOLDERS (Using local paths) ---
# This will create a 'vegetable_dataset' folder right next to your notebook
base_dest_dir = "./vegetable_dataset" 
train_img_dir = osp.join(base_dest_dir, "train", "images")
train_lbl_dir = osp.join(base_dest_dir, "train", "labels")
val_img_dir   = osp.join(base_dest_dir, "val", "images")
val_lbl_dir   = osp.join(base_dest_dir, "val", "labels")

# Create these new folders
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_lbl_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_lbl_dir, exist_ok=True)

# --- 3. GET ALL IMAGE PATHS ---
img_paths = glob.glob(osp.join(source_img_dir, "*.png"))

# Shuffle the list randomly
random.seed(42) # for reproducible results
random.shuffle(img_paths)

# --- 4. CALCULATE SPLIT ---
split_index = int(len(img_paths) * 0.8) # 80% for training
train_paths = img_paths[:split_index]
val_paths   = img_paths[split_index:]

print(f"Found {len(img_paths)} total images.")
print(f"Splitting into {len(train_paths)} training images and {len(val_paths)} validation images.")

# --- 5. FUNCTION TO COPY FILES ---
def copy_files(file_list, img_dest, lbl_dest):
    copied_count = 0
    for img_path in file_list:
        base_filename = osp.basename(img_path)
        base_name = osp.splitext(base_filename)[0]

        lbl_path = osp.join(source_lbl_dir, base_name + ".txt")
        new_img_path = osp.join(img_dest, base_filename)
        new_lbl_path = osp.join(lbl_dest, base_name + ".txt")

        # Create the copies
        if osp.exists(lbl_path): # Check if label exists
            shutil.copy(img_path, new_img_path) # <-- USE SHUTIL.COPY
            shutil.copy(lbl_path, new_lbl_path) # <-- USE SHUTIL.COPY
            copied_count += 1
        else:
            print(f"Warning: No label found for {img_path}")
    print(f"Successfully copied {copied_count} image/label pairs.")

# --- 6. RUN THE SPLIT ---
print("\nCopying training files...")
copy_files(train_paths, train_img_dir, train_lbl_dir)
print("Copying validation files...")
copy_files(val_paths, val_img_dir, val_lbl_dir)

print("---")
print("Data splitting complete!")
print(f"Dataset is ready in: {base_dest_dir}")

Found 619 total images.
Splitting into 495 training images and 124 validation images.

Copying training files...
Successfully copied 495 image/label pairs.
Copying validation files...
Successfully copied 124 image/label pairs.
---
Data splitting complete!
Dataset is ready in: ./vegetable_dataset


In [23]:
%%writefile vegetable_data.yaml

# 1. Define the paths to your NEW train/val folders
path: ./vegetable_dataset
train: ./train/images
val: ./val/images

# 2. Define the number of classes
# (Count how many names were in your classes.txt file)
nc: 7  # <--- EDIT THIS (e.g., 15)

# 3. Define the class names
# (Copy and paste the list from Step 1)
names:
  - lettuce
  - potato
  - carrot
  - onion
  - garlic
  - leek
  - broccoli

Overwriting vegetable_data.yaml


In [3]:
import torch

# This is the most important command:
is_cuda_available = torch.cuda.is_available()

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {is_cuda_available}")

if is_cuda_available:
    device_count = torch.cuda.device_count()
    print(f"CUDA Device Count: {device_count}")
    for i in range(device_count):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. Training will run on CPU.")

PyTorch version: 2.9.0+cu126
CUDA available: True
CUDA Device Count: 1
Device 0: NVIDIA GeForce RTX 4060 Laptop GPU


In [1]:
%pip uninstall torch torchvision -y

%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126

Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torch-2.9.0%2Bcu126-cp311-cp311-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchvision-0.24.0%2Bcu126-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached https://download.pytorch.org/whl/numpy-2.3.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached https://download.pytorch.org/whl/pillow-11.3.0-cp311-cp311-win_amd64.whl.metadata (9.2 kB)
  Usin


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
x = torch.rand(5, 3)
print(x)
torch.cuda.is_available()

tensor([[0.5033, 0.4575, 0.7377],
        [0.4413, 0.3448, 0.4454],
        [0.0668, 0.7359, 0.3817],
        [0.8504, 0.5061, 0.6630],
        [0.3145, 0.5130, 0.6714]])


True

In [1]:
from ultralytics import YOLO

# Load a pre-trained model
model = YOLO('yolov8n.pt')

# Train the model!
results = model.train(data='vegetable_data.yaml', epochs=250, imgsz=640)

Ultralytics 8.3.226  Python-3.11.9 torch-2.9.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=vegetable_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, pl

In [10]:
from ultralytics import YOLO

# 3. Loading the trained model
model = YOLO('runs/detect/train2/weights/best.pt')

results = model.predict(r'C:\Users\Aarin Bhatta\OneDrive\Desktop\Project\CulinaryVisAI\CulinaryVis\cv_model\venv\vegetable_dataset\val\images\IMG_3888.png')

# 5. Showing the results (this will display the image in Colab)
results[0].show()


image 1/1 C:\Users\Aarin Bhatta\OneDrive\Desktop\Project\CulinaryVisAI\CulinaryVis\cv_model\venv\vegetable_dataset\val\images\IMG_3888.png: 640x480 6 garlics, 5.1ms
Speed: 2.0ms preprocess, 5.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)


In [1]:
!pip install roboflow

                

  Using cached roboflow-1.2.11-py3-none-any.whl.metadata (9.7 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached pi_heif-1.1.1-cp311-cp311-win_amd64.whl.metadata (6.7 kB)
  Using cached pillow_avif_plugin-1.5.2-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
Using cached roboflow-1.2.11-py3-none-any.whl (89 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   -------------------------------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Aarin Bhatta\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="ghjNwubjWPfYvYDwrCyr")
project = rf.workspace("yolo-jpkho").project("combined-vegetables-fruits")
version = project.version(8)
dataset = version.download("yolov8")

  Using cached roboflow-1.2.11-py3-none-any.whl.metadata (9.7 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
Using cached roboflow-1.2.11-py3-none-any.whl (89 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Aarin Bhatta\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'roboflow'

In [2]:
from ultralytics import YOLO

# Load your custom-trained model
model = YOLO(r'C:\Users\Aarin Bhatta\OneDrive\Desktop\Project\CulinaryVisAI\CulinaryVis\cv_model\best.pt')

# Your image URL
image_url = 'https://www.bhg.com/thmb/vr6SyEgkmNO0Qu2wxph90gKYey4=/4000x0/filters:no_upscale():strip_icc()/BHG-recipes-how-to-cooking-basics-types-of-onions-hero-55bb0103826149529d4824f69372fa10.jpg'

# Run prediction, but with a lower confidence threshold
results = model.predict(image_url, save=True, conf=0.15)

# Show the results
results[0].show()


WARNING Download failure, retrying 1/3 https://www.bhg.com/thmb/vr6SyEgkmNO0Qu2wxph90gKYey4=/4000x0/filters:no_upscale():strip_icc()/BHG-recipes-how-to-cooking-basics-types-of-onions-hero-55bb0103826149529d4824f69372fa10.jpg... HTTP Error 460: 
image 1/1 c:\Users\Aarin Bhatta\OneDrive\Desktop\Project\CulinaryVisAI\CulinaryVis\cv_model\venv\BHG-recipes-how-to-cooking-basics-types-of-onions-hero-55bb0103826149529d4824f69372fa10.jpg: 448x640 1 apple, 1 cucumber, 21 onions, 3 oranges, 29.6ms
Speed: 2.0ms preprocess, 29.6ms inference, 13.3ms postprocess per image at shape (1, 3, 448, 640)
Results saved to C:\Users\Aarin Bhatta\OneDrive\Desktop\Project\CulinaryVisAI\CulinaryVis\cv_model\venv\runs\detect\predict
